In [2]:
# Import necessary libraries and make necessary arrangements
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

import warnings
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
warnings.filterwarnings('ignore')

In [9]:
df = pd.read_excel("data/orders_by_location.xlsx")

In [10]:
df

,location_id,location_name,date,order_count
0,6.0,"San Francisco, CA",2018-01-01,8411.0
1,19.0,"Santa Barbara, CA",2018-01-01,265.0
2,6.0,"San Francisco, CA",2018-01-02,4886.0
3,19.0,"Santa Barbara, CA",2018-01-02,178.0
4,6.0,"San Francisco, CA",2018-01-03,4505.0
...,...,...,...,...
1293,6.0,"San Francisco, CA",2019-08-10,7934.0
1294,19.0,"Santa Barbara, CA",2019-08-10,263.0
1295,114.0,"Los Angeles, CA",2019-08-11,216.0
1296,6.0,"San Francisco, CA",2019-08-11,7468.0


## HELPER FUNCTIONS

In [12]:
# Check dataframe
def check_df(dataframe, head=5, tail=5, quan=False):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(tail))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())

    if quan:
        print("##################### Quantiles #####################")
        print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [14]:
# Lag/Shifted Features
def lag_features(dataframe, lags):
    for lag in lags:
        dataframe['sales_lag_' + str(lag)] = dataframe.groupby(["store", "item"])['sales'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe

In [15]:
# Rolling Mean Features
def roll_mean_features(dataframe, windows):
    for window in windows:
        dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby(["store", "item"])['sales']. \
                                                          transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(
            dataframe)
    return dataframe

In [16]:
def ewm_features(dataframe, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            dataframe['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                dataframe.groupby(["store", "item"])['sales'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe

## EXPLORATORY DATA ANALYSIS

In [11]:
# Let's check the time periods of train and test sets
df["date"].min(), df["date"].max()  

(Timestamp('2018-01-01 00:00:00'), Timestamp('2019-08-11 00:00:00'))

In [13]:
check_df(df)

##################### Shape #####################
(1298, 4)
##################### Types #####################
location_id             float64
location_name            object
date             datetime64[ns]
order_count             float64
dtype: object
##################### Head #####################
   location_id      location_name       date  order_count
0          6.0  San Francisco, CA 2018-01-01       8411.0
1         19.0  Santa Barbara, CA 2018-01-01        265.0
2          6.0  San Francisco, CA 2018-01-02       4886.0
3         19.0  Santa Barbara, CA 2018-01-02        178.0
4          6.0  San Francisco, CA 2018-01-03       4505.0
##################### Tail #####################
      location_id      location_name       date  order_count
1293          6.0  San Francisco, CA 2019-08-10       7934.0
1294         19.0  Santa Barbara, CA 2019-08-10        263.0
1295        114.0    Los Angeles, CA 2019-08-11        216.0
1296          6.0  San Francisco, CA 2019-08-11       7468

### TODO : Change datatypes for location_id and ordercount

In [19]:
# Checking the total number of location_ids of operations
values = df['location_id'].value_counts(dropna=False).keys().tolist()
counts = df['location_id'].value_counts(dropna=False).tolist()
value_dict = dict(zip(values, counts))
value_dict

{6.0: 588, 19.0: 588, 114.0: 122}

In [21]:
# Checking the total number of location_ids of operations
values = df['location_name'].value_counts(dropna=False).keys().tolist()
counts = df['location_name'].value_counts(dropna=False).tolist()
value_dict = dict(zip(values, counts))
value_dict

{'San Francisco, CA': 588, 'Santa Barbara, CA': 588, 'Los Angeles, CA': 122}

In [22]:
# There is need to clean the location data

In [23]:
# Distribution of Orders
df["order_count"].describe([0.10, 0.30, 0.50, 0.70, 0.80, 0.90, 0.95, 0.99])

count     1298.000000
mean      3618.233436
std       3426.050028
min          0.000000
10%        207.700000
30%        881.600000
50%       1592.500000
70%       5940.900000
80%       6995.000000
90%       8522.800000
95%       9742.200000
99%      11810.870000
max      14673.000000
Name: order_count, dtype: float64

In [25]:
# Sales distribution per location
df.groupby(["location_name"]).agg({"order_count": ["sum"]})

,order_count
,sum
location_name,
"Los Angeles, CA",10178.0
"San Francisco, CA",4150668.0
"Santa Barbara, CA",535621.0


In [26]:
# Sales statistics per location
df.groupby(["location_name"]).agg({"order_count": ["sum", "mean", "median", "std"]})

order_count                                  
                          sum         mean  median          std
location_name                                                  
Los Angeles, CA       10178.0    83.426230    80.5    43.722059
San Francisco, CA   4150668.0  7058.959184  6720.5  1962.626339
Santa Barbara, CA    535621.0   910.920068  1004.5   534.043776